In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
pathstring="~/Dropbox/CDS-2019-AlbanyHub/ToDatabase/TotalUtilities_v02.csv"
# df_array = []
# for i in range(2012, 2020):
#     df_array.append(pd.read_csv(pathstring+str(i)+".csv"))
# df = pd.concat(df_array)
df = pd.read_csv(pathstring)
df.shape

(14748395, 18)

## Questions:
Consumption<0<br>
Very high consumption numbers (19999999)<br>
Negative Charge amounts <br>
Zero Days of Service<br>
Charge with no consumption/consumption with no charge<br>
Missing data<br>
Meaning of Service Rate?
## To Do:
<ul>
<li>Recode units of measure<br>
<li>Decide relevant type categories


In [ ]:
df.head()

# 1. Identifiers

Premise Address, Month, Year, Service Type

# 2. Date Range

2012-2019

# 3. Counts of Observations

In [ ]:
df.shape

# 4. Validate Data

In [ ]:
df.columns

# 5. Inspect Values

#### Negative Consumption Values

In [ ]:
#c_less0= df[df['Consumption']<0]
df.sort_values(by='Consumption').head(20)

Both consumption and charge amount can be less than or equal to 0
#### High Consumption Values

In [ ]:
df.sort_values(by='Consumption', ascending=False).head(20)

Some consumption values seem to be maxed out at 9999999. High values are all DELC =  Demand Commercial Electric(?)
#### Negative Charge Amounts

In [ ]:
df.sort_values(by='Charge Amount').head(20)

Repayments with NaN service types - 300 Pine Ave seems to get monthly repayments

In [ ]:
df.sort_values(by='Charge Amount', ascending=False).head(20)

#### Zero Days of Service

In [ ]:
df.sort_values(by='Days of Service').head(20)

In [ ]:
df.sort_values(by='Days of Service', ascending=False).head(20)

Can have projects with 0 days of service. Consumption/Charge Amount is not always 0

In [ ]:
df[(df['Days of Service']==0)&(df['Charge Amount']>0)]

#### Consumption with no charge

In [ ]:
df[(df['Consumption']>0)&(df['Charge Amount']==0)]

Negative consumption with positive charge, vice versa

In [ ]:
df[(df['Consumption']<0)&(df['Charge Amount']>0)].head()

In [ ]:
df[(df['Consumption']<0)].shape

#### Percent of data which is complete (no nan)

In [ ]:
df2 = df.drop(['City', 'State'], axis=1).dropna()
l1 = df.shape[0]
df2.shape[0]/l1

In [ ]:
df = df.fillna(value=0)

In [ ]:
def recode(series):
    if series == 'Jan':
        return 1
    elif series == 'Feb':
        return 2
    elif series== 'Mar':
        return 3
    elif series == 'Apr':
        return 4
    elif series== 'May':
        return 5
    elif series == 'Jun':
        return 6
    elif series== 'Jul':
        return 7
    elif series == 'Aug':
        return 8
    elif series== 'Sep':
        return 9
    elif series == 'Oct':
        return 10
    elif series== 'Nov':
        return 11
    elif series == 'Dec':
        return 12
df['Month']=df['Month'].apply(recode)

In [ ]:
# Other zero values
df[df['Charge Date'].isnull()].shape

# 6. Temporal Distributions

In [ ]:
df_group = df.groupby(['Service Type', 'Month'])
mns = df_group.mean()
vals = mns.loc['CELC']['Consumption']

In [ ]:
plt.bar(range(12), vals)
plt.title("Average CELC over 12 months")

In [ ]:
vals = mns.loc['CGAS']['Consumption']
plt.bar(range(12), vals)
plt.title("Average CGAS over 12 months")

In [ ]:
vals = mns.loc['RELC']['Consumption']
plt.bar(range(12), vals)
plt.title("Average RELC over 12 months")

# 7. Misc Questions

#### correlation between service type and unit of measure
Service type corresponds to one unit of measure (except a few values in CGAS)<br>
MCF = Thousand Cubic Feet, #CCF = Hundred Cubic Feet<br>
standardize this?<br>

In [ ]:

df_group = df.groupby(['Service Type', 'Unit of Measure'])
df_counts = df_group.count()
df_counts

#### Other way around

In [ ]:

df_group = df.groupby(['Unit of Measure', 'Service Type'])
df_counts = df_group.count()
df_counts

From the data dictionary:<br>
CELC	100-149	Commerical Electric<br>
CGAR	500-599	Commerical Garbage <b>not in data set</b><br> 
CGAS	200-299	Commercial Gas<br>
CO	    200-299	Cash Out<br>
CSWR	400-499	Commercial Sewer<br>
CTEL	800-899	Commercial Telecom <b> not in data set</b><br>
CWTR	300-399	Commercial Water<br>
DELC	100-149	Demand Commercial Electric<br>
FGAS	200-299	Firm Gas<br>
GELC	100-149	Goverment Electric<br>
GGAS	200-299	Goverment Gas<br>
GWTR	300-399	Government Water<br>
IGAS	200-299	Interruptible Gas<br>
MSWR	400-499	Metered Sewer<br>
RELC	100-149	Residential Electric<br>
RGAR	500-599	Residential Garbage<b> not in data set</b><br>
RGAS	200-299	Residential Gas<br>
RSWR	400-499	Residential Sewer<br>
RTEL	800-899	Residential Telecom <b> not in data set</b><br> 
RWTR	300-399	Residential Water<br>
SCLT	150-199	Security Light <b> not in data set</b><br>
SLCM	150-199	Security Light Commercial <b> not in data set</b><br>
SLRE	150-199	Security Light Residential <b> not in data set</b><br>
STRM	700-799	Storm water <b> not in data set</b><br>
TEMP	100-149	Temporary Services<br>
UELC	100-149	Unmetered Electric <b> not in data set</b><br>
USWR	400-499	Unmetered Sewer<br>
WCOM	300-399	Water - Compound Meter<br>

#### Check to see if the service codes correspond

In [ ]:
def get_stats(group):
    return {'max': group.max(), 'min': group.min(), 'count': group.count()}
df['Service Number'].groupby(df['Service Type']).apply(get_stats).unstack()

In [ ]:
df.shape

#### Service Rates

In [ ]:
df_group = df.groupby(['Service Type', 'Service Rate'])
df_counts = df_group.count()
df_counts.iloc[81:120]

In [ ]:
df_group = df.groupby(['Service Rate', 'Service Type'])
df_counts = df_group['Premise Address'].count()
df_counts[50:90]

#### What Service types have consumption less than 0?

In [ ]:
df_cless0 = df.loc[df["Consumption"]<0, :]
df_cless0_gp = df_cless0.groupby(['Service Type'])
df_cless0_gp.count()
#No clear pattern

In [ ]:
df_eq0 = df.loc[df["Consumption"]==0, :]
df_eq0_gp = df_eq0.groupby(['Service Type'])
df_eq0_gp.count()
#No clear pattern

#### Looking at individual projects

In [ ]:
def toDate(series):
    return dt.datetime.strptime(series, '%Y-%m-%d')
def toDate2(series):
    return dt.datetime.strptime(series[:-9], '%Y-%m-%d')
#shift the year and month back by one to account for discrepencies in billing time
def shift_ym(series):
    if series['Month'] == 1:
        return {'YearShift':y-1, 'MonthShift':12}
    else:
        return {'YearShift':y, 'MonthShift':m-1}


In [ ]:
w0 = pd.read_csv("~/Dropbox/CDS-2019-AlbanyHub/ToDatabase/LCD_month.csv")
df.drop(columns=['ChargeID','ChargeID.1', 'ChargeID.1.1'], inplace=True, errors='ignore')
w = w0.loc[w0['YEAR']>=2012, :].copy()
w['DATE2'] = w['DATE'].apply(toDate2)
w.head()

In [ ]:
#o = df[df['Premise Address'] == '2116 W WADDELL AVE']
#o = df[df['Premise Address'] == '1921 EDGERLY AVE'].copy()
o = df[df['Premise Address'] == '912 JEFFERIES AVE']
#o = df[df['Premise Address'] == '923 W GORDON AVE']
o.groupby(by=['Year', 'Service Type']).count()

In [ ]:
sr = o.loc[o['Service Type']=='RELC', :].copy()
sr['Charge Date'] = sr['Charge Date'].apply(toDate)

In [ ]:
p = sr.plot(x='Charge Date', y='Consumption')

In [ ]:
p = w.plot(x='DATE2', y='CLDD')

In [ ]:
#Get consumption and weather normalized by days per month
sr2 = sr.loc[:, ['Year', 'Month', 'Service Type', 'Premise Address', 'Charge Amount', 'Consumption']].groupby(by=['Year', 'Month', 'Service Type', 'Premise Address'])
sr2 = sr2.sum().reset_index()
days_per_month = [31,28,31,30,31,30,31,31,30,31,30,31]
days_per_month_ly =[31,29,31,30,31,30,31,31,30,31,30,31]
w0['HTDD_norm'] = w0['HTDD']
w0['CLDD_norm'] = w0['CLDD']
sr2['Consumption_norm'] = sr2['Consumption']
#Also print months which have more than one record 
for y in range(2001, 2019):
    for m in range(1,13):
        record_w = w0.loc[(w0['YEAR']==y)&(w0['MONTH']==m), :]
        record_u = sr2.loc[(sr2['Year']==y)&(sr2['Month']==m), :]
        if len(record_u)>1:
            print((m, y))
        if y%4 == 0:
            w0.loc[record_w.index, 'HTDD_norm'] = w0.loc[record_w.index, 'HTDD']/days_per_month_ly[m-1]
            w0.loc[record_w.index, 'CLDD_norm'] = w0.loc[record_w.index, 'CLDD']/days_per_month_ly[m-1]
            sr2.loc[record_u.index, 'Consumption_norm'] = sr2.loc[record_u.index, 'Consumption']/days_per_month_ly[m-1]
        else:
            w0.loc[record_w.index, 'HTDD_norm'] = w0.loc[record_w.index, 'HTDD']/days_per_month[m-1]
            w0.loc[record_w.index, 'CLDD_norm'] = w0.loc[record_w.index, 'CLDD']/days_per_month[m-1]
            sr2.loc[record_u.index, 'Consumption_norm'] = sr2.loc[record_u.index, 'Consumption']/days_per_month[m-1]

In [ ]:
new_df = pd.merge(w0, sr2,  how='left', left_on=['YEAR', 'MONTH'], right_on=['Year','Month'])
new_df_sub = new_df.loc[new_df['YEAR']==2013, :]

In [ ]:
new_df_sub.plot(x='CLDD_norm', y='Consumption_norm', kind='scatter')

In [ ]:
sr[sr['Year']==2018]

In [ ]:
df[(df['Year']==2018) & (df['Service Type']=='RGAS')].shape

In [ ]:
df[(df['Year']==2017) & (df['Service Type']=='RGAS')]

In [ ]:
w0